In [1]:
import pandas as pd
import re
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer


In [24]:
data = pd.read_csv('train.csv')

In [3]:
data.head()

,id,label,tweet
0,1,0,@user when a father is dysfunctional and is s...
1,2,0,@user @user thanks for #lyft credit i can't us...
2,3,0,bihday your majesty
3,4,0,#model i love u take with u all the time in ...
4,5,0,factsguide: society now #motivation


In [4]:
data.isnull().sum()

id       0
label    0
tweet    0
dtype: int64

In [5]:
ps = PorterStemmer()

In [25]:
def all_the_cooking(line):
    reviews = re.sub('[^a-zA-Z]' , ' ', line)
    reviews = reviews.lower()
    reviews = reviews.split()
    reviews = [ps.stem(word) for word in reviews if not word in set(stopwords.words('english'))]
    reviews = ' '.join(reviews)
    return(reviews)

In [26]:
for i,line in enumerate(data.tweet):
    data.tweet[i] = all_the_cooking(line)

C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [27]:
data.head()

,id,label,tweet
0,1,0,user father dysfunct selfish drag kid dysfunct...
1,2,0,user user thank lyft credit use caus offer whe...
2,3,0,bihday majesti
3,4,0,model love u take u time ur
4,5,0,factsguid societi motiv


In [28]:
vect = TfidfVectorizer()

In [29]:
tfid_data_x = vect.fit_transform(data['tweet'])


In [32]:
train_y  = data['label']

In [33]:
from imblearn.over_sampling import ADASYN

In [34]:
ad = ADASYN(random_state = 777 , ratio = 1.0)

In [35]:
ad_xtrain_tfid, ad_train_y = ad.fit_sample(tfid_data_x , train_y)

In [38]:
ad_xtrain_tfid.shape

(59146, 31307)

In [39]:
ad_train_y.shape

(59146,)

In [42]:
from sklearn.model_selection import train_test_split

In [43]:
train_x, test_x, train_y, test_y = train_test_split(ad_xtrain_tfid, ad_train_y)

In [45]:
from sklearn.naive_bayes import MultinomialNB
model = MultinomialNB()
model.fit(X=train_x , y = train_y)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [46]:
expected = test_y
predicted=model.predict(test_x)

In [47]:
from sklearn import metrics

print(metrics.classification_report(expected, predicted))
print(metrics.confusion_matrix(expected, predicted))


              precision    recall  f1-score   support

           0       0.98      0.90      0.94      7392
           1       0.91      0.98      0.94      7395

    accuracy                           0.94     14787
   macro avg       0.94      0.94      0.94     14787
weighted avg       0.94      0.94      0.94     14787

[[6661  731]
 [ 127 7268]]


In [49]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(test_y, predicted)

In [50]:
accuracy

0.941976060052749

In [51]:
metrics.f1_score(test_y,predicted)

0.9442639989606341